In [38]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [2]:
op_list=pd.read_csv('../data/opioids.csv')
op_list

,Drug Name,Generic Name
0,ABSTRAL,FENTANYL CITRATE
1,ACETAMINOPHEN-CODEINE,ACETAMINOPHEN WITH CODEINE
2,ACTIQ,FENTANYL CITRATE
3,ASCOMP WITH CODEINE,CODEINE/BUTALBITAL/ASA/CAFFEIN
4,ASPIRIN-CAFFEINE-DIHYDROCODEIN,DIHYDROCODEINE/ASPIRIN/CAFFEIN
...,...,...
108,XODOL 7.5-300,HYDROCODONE/ACETAMINOPHEN
109,XYLON 10,HYDROCODONE/IBUPROFEN
110,ZAMICET,HYDROCODONE/ACETAMINOPHEN
111,ZOHYDRO ER,HYDROCODONE BITARTRATE


In [3]:
overdoses_2014=pd.read_csv('../data/overdoses.csv')
# overdoses_2014

In [4]:
long_df=pd.read_csv('../data/Wide_Master.csv')
long_df

,State,County,Year,Total,Heroin,Other,Methadone,Population,LandArea,Bike,...,Income,Non_US_Born,Bachelor_Degree,Grad_Degree,HS_Grad,Less_Than_HS,Associates_Degree,Unemployment,GDP Total,"GDP Education, Health, Social Assistance"
0,Alabama,Baldwin,2011,10.0,NaN,NaN,NaN,186717.0,1589.78,1.0,...,67770.0,5147.0,22946.0,10923.0,36271.0,15094.0,39216.0,9.0,5016735.0,478277
1,Alabama,Cullman,2011,NaN,NaN,NaN,NaN,80536.0,734.84,NaN,...,51374.0,1582.0,4807.0,2940.0,17883.0,12788.0,16286.0,9.5,2298846.0,207610
2,Alabama,Etowah,2011,NaN,NaN,NaN,NaN,104303.0,534.99,NaN,...,51330.0,1833.0,6642.0,4270.0,22939.0,13064.0,24325.0,10.0,2801621.0,430563
3,Alabama,Jefferson,2011,39.0,10.0,28.0,10.0,658931.0,1111.28,0.1,...,66409.0,20934.0,78798.0,48016.0,121313.0,58142.0,131726.0,9.3,39515681.0,4542034
4,Alabama,Madison,2011,14.0,NaN,12.0,NaN,340111.0,801.59,0.3,...,76342.0,14164.0,50973.0,30157.0,48925.0,24067.0,62801.0,8.3,19512201.0,1135690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5407,Wisconsin,Washington,2017,18.0,NaN,12.0,NaN,135101.0,430.70,0.1,...,89470.0,2956.0,20316.0,7526.0,28297.0,5231.0,32364.0,2.8,5636773.0,476010
5408,Wisconsin,Waukesha,2017,52.0,27.0,27.0,NaN,400621.0,549.57,0.1,...,105275.0,17376.0,78511.0,40693.0,65576.0,11104.0,81873.0,2.9,28028503.0,2927629
5409,Wisconsin,Winnebago,2017,16.0,NaN,NaN,NaN,170414.0,434.49,0.4,...,71572.0,4719.0,22213.0,8838.0,39008.0,8679.0,34896.0,3.0,9274302.0,649081
5410,Wyoming,Fremont,2017,NaN,NaN,NaN,NaN,39803.0,9183.81,NaN,...,70101.0,469.0,4158.0,2073.0,8497.0,2381.0,9672.0,5.5,1595939.0,138479


In [5]:
long_df=long_df.fillna(0)
long_df.to_csv('../data/Wide_Master_clean.csv')


In [69]:
prescriptions=pd.read_csv('../data/prescriber-info.csv')
prescriptions

,NPI,Gender,State,Credentials,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,...,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber
0,1710982582,M,TX,DDS,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1245278100,F,AL,MD,General Surgery,0,0,0,0,0,...,0,0,0,0,0,0,0,0,35,1
2,1427182161,F,NY,M.D.,General Practice,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
3,1669567541,M,AZ,MD,Internal Medicine,0,43,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1679650949,M,NV,M.D.,Hematology/Oncology,0,0,0,0,0,...,0,0,0,0,17,28,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,1619019353,F,WA,M.D.,Family Practice,0,12,0,0,0,...,0,0,0,0,39,0,0,0,0,1
24996,1780693671,F,MI,DO,Internal Medicine,0,15,0,0,0,...,0,0,0,0,20,0,0,0,0,1
24997,1346270956,M,AZ,MD,Emergency Medicine,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
24998,1023116514,F,IN,M.D.,Family Practice,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [70]:
col_list=prescriptions.columns
new_col=[]
for el in col_list:
    new_col.append(el.replace('.'," "))
prescriptions.columns=new_col


In [60]:
mydict={}
for drug in op_list['Drug Name']:
    for pre in prescriptions.columns:
        if pre==drug:
            print(f'yes, {pre} is a match')
            mydict[pre]=prescriptions[pre]

yes, FENTANYL is a match
yes, HYDROMORPHONE HCL is a match
yes, HYDROMORPHONE HCL is a match
yes, METHADONE HCL is a match
yes, MORPHINE SULFATE is a match
yes, MORPHINE SULFATE is a match
yes, MORPHINE SULFATE ER is a match
yes, OXYCODONE HCL is a match
yes, OXYCONTIN is a match
yes, TRAMADOL HCL is a match


In [76]:
gens=op_list['Generic Name'].tolist()
brands=op_list['Drug Name'].tolist()
extras=['NPI', 'Gender', 'State', 'Credentials', 'Specialty','Opioid Prescriber']
allnames=gens+brands+extras
# allnames

In [83]:
pandas_df = prescriptions[prescriptions.columns.intersection(allnames)]
tester=[]

In [85]:
for el in pandas_df['Opioid Prescriber'].tolist():
    if el==1:
        tester.append(True)
    else:
        tester.append(False)

In [88]:
pandas_df['Opioid Prescriber']=tester
pandas_df

C:\Users\A\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,NPI,Gender,State,Credentials,Specialty,FENTANYL,FENTANYL,FENTANYL,FENTANYL,HYDROMORPHONE HCL,...,OXYCODONE HCL,OXYCONTIN,TRAMADOL HCL,TRAMADOL HCL,TRAMADOL HCL,TRAMADOL HCL,TRAMADOL HCL,TRAMADOL HCL,TRAMADOL HCL,Opioid Prescriber
0,1710982582,M,TX,DDS,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
1,1245278100,F,AL,MD,General Surgery,0,0,0,0,0,...,87,0,36,36,36,36,36,36,36,True
2,1427182161,F,NY,M.D.,General Practice,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
3,1669567541,M,AZ,MD,Internal Medicine,0,0,0,0,0,...,29,0,90,90,90,90,90,90,90,True
4,1679650949,M,NV,M.D.,Hematology/Oncology,22,22,22,22,0,...,22,0,0,0,0,0,0,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,1619019353,F,WA,M.D.,Family Practice,0,0,0,0,0,...,21,0,0,0,0,0,0,0,0,True
24996,1780693671,F,MI,DO,Internal Medicine,0,0,0,0,0,...,0,0,29,29,29,29,29,29,29,True
24997,1346270956,M,AZ,MD,Emergency Medicine,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
24998,1023116514,F,IN,M.D.,Family Practice,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True


In [89]:
pandas_df.to_csv('prescriber_data_clean.csv')

In [103]:
sql_test=pd.read_csv('prescriber_data_clean.csv')

In [130]:
sql_test2=pd.read_csv('../data/Wide_Master_clean.csv')
sql_test3=pd.read_csv('../data/overdoses.csv')

In [105]:
sql_test.drop('Unnamed: 0',axis=1,inplace=True)
sql_test

,NPI,Gender,State,Credentials,Specialty,FENTANYL,FENTANYL.1,FENTANYL.2,FENTANYL.3,HYDROMORPHONE HCL,...,OXYCODONE HCL.5,OXYCONTIN,TRAMADOL HCL,TRAMADOL HCL.1,TRAMADOL HCL.2,TRAMADOL HCL.3,TRAMADOL HCL.4,TRAMADOL HCL.5,TRAMADOL HCL.6,Opioid Prescriber
0,1710982582,M,TX,DDS,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
1,1245278100,F,AL,MD,General Surgery,0,0,0,0,0,...,87,0,36,36,36,36,36,36,36,True
2,1427182161,F,NY,M.D.,General Practice,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
3,1669567541,M,AZ,MD,Internal Medicine,0,0,0,0,0,...,29,0,90,90,90,90,90,90,90,True
4,1679650949,M,NV,M.D.,Hematology/Oncology,22,22,22,22,0,...,22,0,0,0,0,0,0,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,1619019353,F,WA,M.D.,Family Practice,0,0,0,0,0,...,21,0,0,0,0,0,0,0,0,True
24996,1780693671,F,MI,DO,Internal Medicine,0,0,0,0,0,...,0,0,29,29,29,29,29,29,29,True
24997,1346270956,M,AZ,MD,Emergency Medicine,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
24998,1023116514,F,IN,M.D.,Family Practice,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True


In [106]:
conn_string='tester:taylor@localhost:5432/Opiate_Analysis'
engine= create_engine(f'postgresql://{conn_string}')

In [107]:
sql_test.to_sql(name='Prescriber_info', con=engine, if_exists='append', index=True)

In [55]:
sql_test2.to_sql(name='Wide_data', con=engine, if_exists='append', index=True)

In [133]:
nums1=[]
pops=[]
for num in sql_test3['Deaths']:
    nums1.append(int(num.replace(',','')))
for num in sql_test3['Population']:
    pops.append(int(num.replace(',','')))

In [134]:
sql_test3['Deaths']=nums1
sql_test3["Population"]=pops

In [136]:
sql_test3.to_csv('../data/overdoses_clean.csv')

In [137]:
sql_test3.to_sql(name='Overdoses', con=engine, if_exists='append', index=True)